# ETL with PySpark

[PySpark](https://spark.apache.org/docs/latest/api/python/) is the Python interface to the analytics engine [Spark](https://spark.apache.org/).
Like `pandas`, it organises data using `DataFrame`s and provides tools to analyse and visualise them.

In [1]:
from pyspark.sql import functions as F, types as T, SparkSession

Before we can use PySpark, we need to initialise a new `SparkSession` which is used to create and interact with `DataFrame`s.

In [2]:
spark = SparkSession.builder\
        .appName("ETL with Spark")\
        .getOrCreate()

We'll use PySpark to explore the [student performance data](https://archive.ics.uci.edu/ml/datasets/Student+Performance) made available by Paulo Cortez.
We start by loading the data.

In [3]:
students = spark.read\
           .option("header", True)\
           .option("inferSchema", True)\
           .csv("datasets/student_performance.csv")

Spark `DataFrame`s share many similarities with `pandas` `DataFrame`s such as the `columns` and `dtypes` attributes.

In [4]:
students.columns

['school',
 'sex',
 'age',
 'home_area',
 'family_size',
 'parents_cohabit',
 'education_mother',
 'education_father',
 'occupation_mother',
 'occupation_father',
 'reason',
 'guardian',
 'travel_time',
 'study_time',
 'failures',
 'extra_school_support',
 'family_support',
 'extra_tutoring',
 'extracurricular_activities',
 'nursery',
 'higher_education',
 'internet_access',
 'romantic_relationship',
 'family_relationships_quality',
 'free_time',
 'going_out',
 'alcohol_weekdays',
 'alcohol_weekend',
 'health_status',
 'absences',
 'final_grade']

In [5]:
students.dtypes

[('school', 'string'),
 ('sex', 'string'),
 ('age', 'int'),
 ('home_area', 'string'),
 ('family_size', 'string'),
 ('parents_cohabit', 'boolean'),
 ('education_mother', 'string'),
 ('education_father', 'string'),
 ('occupation_mother', 'string'),
 ('occupation_father', 'string'),
 ('reason', 'string'),
 ('guardian', 'string'),
 ('travel_time', 'string'),
 ('study_time', 'string'),
 ('failures', 'string'),
 ('extra_school_support', 'boolean'),
 ('family_support', 'boolean'),
 ('extra_tutoring', 'boolean'),
 ('extracurricular_activities', 'boolean'),
 ('nursery', 'boolean'),
 ('higher_education', 'boolean'),
 ('internet_access', 'boolean'),
 ('romantic_relationship', 'boolean'),
 ('family_relationships_quality', 'int'),
 ('free_time', 'int'),
 ('going_out', 'int'),
 ('alcohol_weekdays', 'int'),
 ('alcohol_weekend', 'int'),
 ('health_status', 'int'),
 ('absences', 'int'),
 ('final_grade', 'int')]

One important difference is that Spark often uses **lazy evaluation**.
This means that commands will return immediately but won't execute unless an action is triggered.

For example, the number of rows in our `DataFrame` is computed only when required.

In [6]:
students.count()

649

## Selecting rows and columns

### Selecting rows

You can select rows that satisfy certain conditions using `filter`.

In [7]:
students.filter(F.col("sex") == "M").count()

266

Multiple conditions can be combined using logical operators.

In [8]:
students.filter(
    (F.col("sex") == "M") &
    (F.col("age") >= 18)
).count()

73

### Selecting columns

You can select specific columns by name using `select`.

In [9]:
students.select(F.col("school"), F.col("sex"), F.col("age")).show(5)

+---------------+---+---+
|         school|sex|age|
+---------------+---+---+
|Gabriel Pereira|  F| 18|
|Gabriel Pereira|  F| 17|
|Gabriel Pereira|  F| 15|
|Gabriel Pereira|  F| 15|
|Gabriel Pereira|  F| 16|
+---------------+---+---+
only showing top 5 rows



### Selecting rows and columns simultaneously

The functions `filter` and `select` can be combined to simultaneously select rows and columns.

In [10]:
students.select(F.col("school"), F.col("age"))\
        .filter(F.col("sex") == "M")\
        .show(5)

+---------------+---+
|         school|age|
+---------------+---+
|Gabriel Pereira| 16|
|Gabriel Pereira| 16|
|Gabriel Pereira| 15|
|Gabriel Pereira| 15|
|Gabriel Pereira| 15|
+---------------+---+
only showing top 5 rows



### Using the Spark SQL interface

Alternatively, you can register the `DataFrame` as a temporary view and issue [Spark SQL](https://spark.apache.org/docs/latest/api/sql/) queries using `sql`.

In [11]:
students.createOrReplaceTempView("students")

In [12]:
spark.sql("""
    SELECT school, age
    FROM students
    WHERE sex = 'M'
""").show(5)

+---------------+---+
|         school|age|
+---------------+---+
|Gabriel Pereira| 16|
|Gabriel Pereira| 16|
|Gabriel Pereira| 15|
|Gabriel Pereira| 15|
|Gabriel Pereira| 15|
+---------------+---+
only showing top 5 rows



## Split-apply-combine

Most transformations in Spark are achieved using the split-apply-combine paradigm provided by `groupBy`:

1. First, observations are split into groups defined by one or more categorical variables
1. Then, a summary function such as `mean()` is applied within each group independently
1. Finally, results are combined back into a `DataFrame`

![Split-apply-combine](http://i.imgur.com/yjNkiwL.png)

In [13]:
students.groupBy(F.col("school"))\
        .agg(F.avg((F.col('sex') == "M").cast(T.IntegerType())).alias("pct_men"),
             F.avg(F.col("age")).alias("avg_age"))\
        .show()

+--------------------+-------------------+------------------+
|              school|            pct_men|           avg_age|
+--------------------+-------------------+------------------+
|Mouzinho da Silveira|0.35398230088495575| 16.88938053097345|
|     Gabriel Pereira| 0.4397163120567376|16.666666666666668|
+--------------------+-------------------+------------------+



In [14]:
spark.sql("""
    SELECT
        school,
        AVG(CAST(sex = 'M' AS int)) AS pct_men,
        AVG(age) AS avg_age
    FROM students
    GROUP BY school
""").show()

+--------------------+-------------------+------------------+
|              school|            pct_men|           avg_age|
+--------------------+-------------------+------------------+
|Mouzinho da Silveira|0.35398230088495575| 16.88938053097345|
|     Gabriel Pereira| 0.4397163120567376|16.666666666666668|
+--------------------+-------------------+------------------+

